# 📊 Pandas 2 – Ejercicios resueltos y explicados

Este cuaderno está pensado **para repasar justo antes del examen**.

👉 Incluye:
- Explicaciones claras en **Markdown**
- Ejercicios **tipo examen**
- Código comentado
- Incapié en los puntos que **casi seguro entran**

---
## Contenidos clave del examen
- `query`
- Limpieza de datos (nulos, duplicados)
- Transformaciones (`map`, `replace`)
- Discretización (`cut`, `qcut`)
- Variables dummy
- Combinación de datasets (`merge`, `concat`, `join`)
- `groupby` y agregaciones
- Cadenas de texto (`str`)


## 1️⃣ Dataset de ejemplo
Creamos un DataFrame que usaremos en casi todos los ejercicios.

In [1]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    'nombre': ['Ana', 'Luis', 'María', 'Luis', 'Juan', 'Ana'],
    'edad': [23, 35, np.nan, 35, 120, 23],
    'ciudad': ['Madrid', 'Sevilla', 'Madrid', 'Sevilla', None, 'Madrid'],
    'salario': [2000, 2500, 1800, 2500, 100000, 2000]
})

df

,nombre,edad,ciudad,salario
0,Ana,23.0,Madrid,2000
1,Luis,35.0,Sevilla,2500
2,María,NaN,Madrid,1800
3,Luis,35.0,Sevilla,2500
4,Juan,120.0,None,100000
5,Ana,23.0,Madrid,2000


## 2️⃣ Consultas con `query` (MUY IMPORTANTE)
⚠️ Suele caer seguro en el examen.

### Ejercicio:
Seleccionar las personas con salario mayor a 2000 y ciudad Madrid.

In [8]:
# Que no salgan repetidos
df.query('salario >= 2000 and ciudad == "Madrid"').drop_duplicates()

,nombre,edad,ciudad,salario
0,Ana,23.0,Madrid,2000


🔑 **Claves de examen:**
- Strings con comillas dobles
- Operadores lógicos: `and`, `or`, `~`
- Acceso a listas con `@lista`


## 3️⃣ Datos ausentes (nulos)

### Comprobar nulos

In [9]:
df.isnull()

,nombre,edad,ciudad,salario
0,False,False,False,False
1,False,False,False,False
2,False,True,False,False
3,False,False,False,False
4,False,False,True,False
5,False,False,False,False


### Eliminar filas con nulos

In [10]:
df.dropna()

,nombre,edad,ciudad,salario
0,Ana,23.0,Madrid,2000
1,Luis,35.0,Sevilla,2500
3,Luis,35.0,Sevilla,2500
5,Ana,23.0,Madrid,2000


### Rellenar nulos con la media

In [11]:
df['edad'] = df['edad'].fillna(df['edad'].mean())
df

,nombre,edad,ciudad,salario
0,Ana,23.0,Madrid,2000
1,Luis,35.0,Sevilla,2500
2,María,47.2,Madrid,1800
3,Luis,35.0,Sevilla,2500
4,Juan,120.0,None,100000
5,Ana,23.0,Madrid,2000


⚠️ **Truco de examen**: `mean()` ignora los nulos automáticamente.

## 4️⃣ Duplicados

In [12]:
df.duplicated()

0    False
1    False
2    False
3     True
4    False
5     True
dtype: bool

In [13]:
df.drop_duplicates()

,nombre,edad,ciudad,salario
0,Ana,23.0,Madrid,2000
1,Luis,35.0,Sevilla,2500
2,María,47.2,Madrid,1800
4,Juan,120.0,None,100000


## 5️⃣ Transformación con `map` y `replace`

In [14]:
map_ciudad = {'Madrid': 'MD', 'Sevilla': 'SV'}
df['ciudad_cod'] = df['ciudad'].map(map_ciudad)
df

,nombre,edad,ciudad,salario,ciudad_cod
0,Ana,23.0,Madrid,2000,MD
1,Luis,35.0,Sevilla,2500,SV
2,María,47.2,Madrid,1800,MD
3,Luis,35.0,Sevilla,2500,SV
4,Juan,120.0,None,100000,NaN
5,Ana,23.0,Madrid,2000,MD


## 6️⃣ Discretización (`cut` y `qcut`)

Convertimos edades en grupos.

In [15]:
df['grupo_edad'] = pd.cut(df['edad'], bins=[0,18,30,65,150], labels=['Menor','Joven','Adulto','Mayor'])
df

,nombre,edad,ciudad,salario,ciudad_cod,grupo_edad
0,Ana,23.0,Madrid,2000,MD,Joven
1,Luis,35.0,Sevilla,2500,SV,Adulto
2,María,47.2,Madrid,1800,MD,Adulto
3,Luis,35.0,Sevilla,2500,SV,Adulto
4,Juan,120.0,None,100000,NaN,Mayor
5,Ana,23.0,Madrid,2000,MD,Joven


📌 **cut** → intervalos fijos
📌 **qcut** → mismo número de datos por grupo

## 7️⃣ Variables dummy

In [16]:
pd.get_dummies(df['grupo_edad'], prefix='edad')

,edad_Menor,edad_Joven,edad_Adulto,edad_Mayor
0,False,True,False,False
1,False,False,True,False
2,False,False,True,False
3,False,False,True,False
4,False,False,False,True
5,False,True,False,False


## 8️⃣ Combinación de datasets (`merge`)

In [17]:
df_salud = pd.DataFrame({
    'nombre': ['Ana', 'Luis', 'María'],
    'seguro': ['Sí', 'No', 'Sí']
})

pd.merge(df, df_salud, on='nombre', how='left')

,nombre,edad,ciudad,salario,ciudad_cod,grupo_edad,seguro
0,Ana,23.0,Madrid,2000,MD,Joven,Sí
1,Luis,35.0,Sevilla,2500,SV,Adulto,No
2,María,47.2,Madrid,1800,MD,Adulto,Sí
3,Luis,35.0,Sevilla,2500,SV,Adulto,No
4,Juan,120.0,None,100000,NaN,Mayor,NaN
5,Ana,23.0,Madrid,2000,MD,Joven,Sí


⚠️ Pregunta típica: diferencia entre `inner`, `left`, `right`, `outer`.

## 9️⃣ GroupBy y agregaciones (CLAVE)

Calcular salario medio por ciudad.

In [18]:
df.groupby('ciudad')['salario'].mean()

ciudad
Madrid     1933.333333
Sevilla    2500.000000
Name: salario, dtype: float64

### Varias funciones a la vez

In [19]:
df.groupby('ciudad').agg({'salario':['mean','max'], 'edad':'mean'})

salario             edad
                mean   max       mean
ciudad                               
Madrid   1933.333333  2000  31.066667
Sevilla  2500.000000  2500  35.000000

## 🔟 Cadenas de texto (`str`)

In [20]:
df['nombre'].str.upper()

0      ANA
1     LUIS
2    MARÍA
3     LUIS
4     JUAN
5      ANA
Name: nombre, dtype: object

## ✅ Consejos finales de examen
- Si ves **limpieza** → piensa en `isnull`, `dropna`, `fillna`
- Si ves **categorías** → `cut`, `qcut`, `get_dummies`
- Si ves **resúmenes** → `groupby`
- Si ves **consultas** → `query`

💪 Si dominas este cuaderno, vas MUY bien preparado.